In [1]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 1.7 MB/s eta 0:00:001.6 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.5 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 1.5 MB/s eta 0:00:000:00:01m eta 0:00:01


In [2]:
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 2.2 MB/s eta 0:00:002.5 MB/s eta 0:00:01


In [82]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [83]:
openai_api_key=os.getenv('OPENAI_API_KEY')

In [84]:
from llama_index.llms.openai import OpenAI

In [51]:
response = OpenAI().complete("Paul Graham is ")
print(response)

a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his work on Lisp, a programming language. Graham has also written several influential essays on startups, entrepreneurship, and technology.


In [85]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

# Using a local LLM

In [86]:
# from llama_index.llms.ollama import Ollama
# from llama_index.core import Settings

# Settings.llm = Ollama(model="llama2", request_timeout=60.0)

# Loading Data (Ingestion)

This ingestion pipeline typically consists of three main stages:

Load the data


Transform the data


Index and store the data

## Loaders

### 1. Loading using SimpleDirectoryReader

The easiest reader to use is our SimpleDirectoryReader, which creates documents out of every file in a given directory. It is built in to LlamaIndex and can read a variety of formats including Markdown, PDFs, Word documents, PowerPoint decks, images, audio and video.

In [87]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()

### 2.Using Readers from LlamaHub

In [22]:
!pip install llama-index-readers-database

In [54]:
# from llama_index.core import download_loader

# from llama_index.readers.database import DatabaseReader

# reader = DatabaseReader(
#     scheme=os.getenv("DB_SCHEME"),
#     host=os.getenv("DB_HOST"),
#     port=os.getenv("DB_PORT"),
#     user=os.getenv("DB_USER"),
#     password=os.getenv("DB_PASS"),
#     dbname=os.getenv("DB_NAME"),
# )

# query = "SELECT * FROM users"
# documents = reader.load_data(query=query)

### 3.Creating Documents directly

In [88]:
from llama_index.core import Document

doc = Document(text="./data/state_of_the_union.txt")

## Transformations

### 1.High-Level Transformation API

Indexes have a .from_documents() method which accepts an array of Document objects and will correctly parse and chunk them up. However, sometimes you will want greater control over how your documents are split up.

In [89]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents([doc])
vector_index.as_query_engine()

If you want to customize core components, like the text splitter, through this abstraction you can pass in a custom transformations list or apply to the global Settings:




In [90]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

# global
from llama_index.core import Settings

Settings.text_splitter = text_splitter

# per-index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter]
)

### 2.Lower-Level Transformation API

key step to process your documents is to split them into "chunks"/Node objects. The key idea is to process your data into bite-sized pieces that can be retrieved / fed to the LLM.

LlamaIndex has support for a wide range of text splitters, ranging from paragraph/sentence/token based splitters to file-based splitters like HTML, JSON.

In [91]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter

documents = SimpleDirectoryReader("./data").load_data()

pipeline = IngestionPipeline(transformations=[TokenTextSplitter()])

nodes = pipeline.run(documents=documents)

In [92]:
from llama_index.embeddings.openai import OpenAIEmbedding

# global
Settings.embed_model = OpenAIEmbedding()

# per-index
index = VectorStoreIndex.from_documents(documents, embed_model=OpenAIEmbedding())

In [44]:
# To save cost we can use huggingface embeddings also
# !pip install llama-index-embeddings-huggingface

In [43]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en-v1.5"
# )

# Getting Started

In [93]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# global default
Settings.embed_model = OpenAIEmbedding()

documents = SimpleDirectoryReader("./data").load_data()

index = VectorStoreIndex.from_documents(documents,embed_model=OpenAIEmbedding())

In [94]:
query_engine = index.as_query_engine()

response = query_engine.query("What is attention ?")

In [95]:
response.response

'Attention is a mechanism used in sequence transduction models that allows the model to focus on specific parts of the input sequence when generating the output sequence. It helps the model to weigh the importance of different input elements during the decoding process.'

# Storing

Once you have data loaded and indexed, you will probably want to store it to avoid the time and cost of re-indexing it. By default, your indexed data is stored only in memory.

## Persisting to disk

In [96]:
index.storage_context.persist(persist_dir="./database")

The simplest way to store your indexed data is to use the built-in .persist() method of every Index, which writes all the data to disk at the location specified. This works for any type of index.

In [97]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./database")

# load index
index = load_index_from_storage(storage_context)

**Important:** if you had initialized your index with a custom transformations, embed_model, etc., you will need to pass in the same options during load_index_from_storage, or have it set as the global settings.



## Using Vector Stores

In indexing, one of the most common types of Index is the VectorStoreIndex. The API calls to create the embeddings in a VectorStoreIndex can be expensive in terms of time and money, so you will want to store them to avoid having to constantly re-index things.

LlamaIndex supports a huge number of vector stores which vary in architecture, complexity and cost. In this example we'll be using Chroma, an open-source vector store.

In [74]:
!pip install chromadb

In [76]:
!pip install llama-index-vector-stores-chroma

In [98]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [107]:
# load some documents
documents = SimpleDirectoryReader("./data").load_data()

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

# create a query engine and query
query_engine = index.as_query_engine()
response = query_engine.query("What is the meaning of attention?")
print(response)

The meaning of attention refers to the mechanism in a model that allows it to focus on specific parts of the input data during processing. This mechanism helps the model to weigh the importance of different elements in the input when making predictions or generating outputs.


# Querying

In [ ]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# build index
index = VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# query
response = query_engine.query("What did the author do growing up?")
print(response)

In this example, we customize our retriever to use a different number for top_k and add a post-processing step that requires that the retrieved nodes reach a minimum similarity score to be included. This would give you a lot of data when you have relevant results but potentially no data if you have nothing relevant.
